In [ ]:
import random
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential,Model
from keras.layers import Dense,Activation,Dropout,Input, BatchNormalization
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split

In [ ]:
#--matplotlib
import matplotlib
matplotlib.use('Agg')
#matplotlib.rcParams['text.latex.preamble']=[r"\usepackage{amsmath}"]
#matplotlib.rc('text',usetex=True)
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.lines import Line2D
import pylab as py


import os
import pylab as py

from tools.tools import checkdir,load,save

# Generating the Models

In [ ]:
path2nptabs='nptabs/'
path2nnmodels='nnmodels/'

In [ ]:
def get_data(k,channel,flavor,path2nptabs,original=False):
    data=np.load('%s/%s/%s/%s.npy'%(path2nptabs,k,channel,flavor))

    num_inputs=3
    num_outputs=2
    #--split into input (x) and output (y)
    x=data[:num_inputs]
    y=data[num_inputs:]

    x,y=x.T,y.T
    x_train_orig,x_test,y_train_orig,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
    xsc=MinMaxScaler()
    ysc=MinMaxScaler()
    x_train=xsc.fit_transform(x_train_orig)
    x_test=xsc.transform(x_test)
    y_train=ysc.fit_transform(y_train_orig)
    y_test=ysc.transform(y_test)

    if original: return x_train,x_test,y_train,y_test,x_train_orig,y_train_orig
    else: return x_train,x_test,y_train,y_test

In [ ]:
def gen_model(EPOCHS, BATCH_SIZE, lr, k, channel, flavor, path2nptabs, path2nnmodels):

    print('\ngenerating model for %s, %s'%(channel,flavor))

    x_train,x_test,y_train,y_test,x_train_orig,y_train_orig = get_data(k,channel,flavor,path2nptabs,original=True)

    input=Input(shape= x_train[0].shape)
    x=Dense(120, activation='relu')(input)
    x = Dropout(0.01)(x)
    x=Dense(120, activation='relu')(x)
    x = Dropout(0.01)(x)
    x=Dense(120, activation='relu')(x)
    x = Dropout(0.01)(x)
    output=Dense(2)(x)
    model=Model(input,output)
    model.summary()
    model_optimizer=Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.00001) 
    model.compile(optimizer=model_optimizer,loss='mean_squared_error',metrics=['accuracy'])
    history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.2, verbose=1)

    checkdir('%s/%s/%s'%(path2nnmodels,k,channel))
    filename='%s/%s/%s/%s.json'%(path2nnmodels,k,channel,flavor)
    save({'model':model,'history':history,'x_train_orig':x_train_orig,'y_train_orig':y_train_orig},filename)

In [ ]:
def plot_predictions(path2nnmodels,k,channel,flavor,path2nptabs):
 
    x_train,x_test,y_train,y_test= get_data(k,channel,flavor,path2nptabs)

    y_test=y_test.T
    y_train=y_train.T

    #--load model data
    modeldata=load('%s/%s/%s/%s.json'%(path2nnmodels,k,channel,flavor))
    model = modeldata['model']

    y_pred=model.predict(x_test).T

    nrows, ncols = 1, 2
    fig=plt.figure(figsize=(ncols*7,nrows*4))
    ax1=plt.subplot(nrows, ncols, 1)
    ax2=plt.subplot(nrows, ncols, 2)

    ax1.scatter(y_test[0],y_pred[0],label='Predicted',s=0.5)
    ax1.plot(y_test[0], y_test[0], color='orange')
    ax1.legend()
    ax1.set_xlabel('Re($\sigma$)_test')
    ax1.set_ylabel('Re($\sigma$)_pred')

    ax2.scatter(y_test[1],y_pred[1],label='Predicted',s=0.5)
    ax2.plot(y_test[1], y_test[1], color='orange')
    ax2.legend()
    ax2.set_xlabel('Im($\sigma$)_test')
    ax2.set_ylabel('Im($\sigma$)_pred')
    checkdir('gallery/%s/%s/%s'%(k,channel,flavor))
    filename='gallery/%s/%s/%s/predictions.png'%(k,channel,flavor)
    plt.savefig(filename)
    plt.close()
    print('Saving figure to %s'%filename)


    y_pred=model.predict(x_train).T

    nrows, ncols = 1, 2
    fig=plt.figure(figsize=(ncols*7,nrows*4))
    ax1=plt.subplot(nrows, ncols, 1)
    ax2=plt.subplot(nrows, ncols, 2)

    ax1.scatter(y_train[0],y_pred[0],label='Predicted',s=0.5)
    ax1.plot(y_train[0], y_train[0], color='orange')
    ax1.legend()
    ax1.set_xlabel('Re($\sigma$)_train')
    ax1.set_ylabel('Re($\sigma$)_pred')

    ax2.scatter(y_train[1],y_pred[1],label='Predicted',s=0.5)
    ax2.plot(y_train[1], y_train[1], color='orange')
    ax2.legend()
    ax2.set_xlabel('Im($\sigma$)_train')
    ax2.set_ylabel('Im($\sigma$)_pred')
    checkdir('gallery/%s/%s/%s'%(k,channel,flavor))
    filename='gallery/%s/%s/%s/trainingset.png'%(k,channel,flavor)
    plt.savefig(filename)
    plt.close()
    print('Saving figure to %s'%filename)

In [ ]:
#--generate the model for 30001
k=30001
epochs=50

channel='qA,qbB'
for flav in ['ub','db','sb','cb','bb']:
    gen_model(epochs,32,1e-4,k,channel,flav,path2nptabs,path2nnmodels)
    plot_metrics(path2nnmodels,k,channel,flav)
    plot_predictions(path2nnmodels,k,channel,flav,path2nptabs)
channel='qbA,qB'
for flav in ['u','d','s','c','b']:
    gen_model(epochs,32,1e-4,k,channel,flav,path2nptabs,path2nnmodels)
    plot_metrics(path2nnmodels,k,channel,flav)
    plot_predictions(path2nnmodels,k,channel,flav,path2nptabs)
channel='qA,gB'
for flav in ['g']:
    gen_model(epochs,32,1e-4,k,channel,flav,path2nptabs,path2nnmodels)
    plot_metrics(path2nnmodels,k,channel,flav)
    plot_predictions(path2nnmodels,k,channel,flav,path2nptabs)
channel='gA,qB'
for flav in ['u','d','s','c','b','ub','db','sb','cb','bb']:
    gen_model(epochs,32,1e-4,k,channel,flav,path2nptabs,path2nnmodels)
    plot_metrics(path2nnmodels,k,channel,flav)
    plot_predictions(path2nnmodels,k,channel,flav,path2nptabs)

# Comparing with actual cross-sections

In [ ]:
#--from tools
from tools.config import conf,load_config
from tools import reader

#--from qcdlib
from qcdlib import aux,eweak,alphaS,mellin

#--from dy
from dy import theory,reader,fakepdf

In [ ]:
conf['order']='NLO'
conf['aux']=aux.AUX()
conf['Q20']=conf['aux'].mc2
conf['eweak']=eweak.EWEAK()
conf['alphaS']=alphaS.ALPHAS()
conf['mellin-pion']=mellin.MELLIN()
conf['mellin']=conf['mellin-pion']

conf['pdfA']=fakepdf.FAKEPDF()
conf['pdf-pion']=conf['pdfA']
conf['pdfB']=fakepdf.FAKEPDF()

In [ ]:
conf['datasets']={}
conf['datasets']['dy-pion']={}
conf['datasets']['dy-pion']['filters']=[]
conf['datasets']['dy-pion']['filters'].append("Q2>4.16**2")
conf['datasets']['dy-pion']['filters'].append("Q2<8.34**2")
conf['datasets']['dy-pion']['filters'].append("xF>0")
conf['datasets']['dy-pion']['filters'].append("xF<0.9")
conf['datasets']['dy-pion']['xlsx']={}
conf['datasets']['dy-pion']['xlsx'][30001]='30001.xlsx'

conf['dy-pion tabs']=reader.READER().load_data_sets('dy-pion')
conf['path2ml4jam']='./'

In [ ]:
conf['dy-pion']=theory.DY_PION()

conf['path2dytab-hybrid']='melltabs/'
conf['path2nntabs']='nnmodels/'

In [ ]:
def get_xspace(k,part,flav=None):
    xspace=np.zeros(len(conf['dy-pion tabs'][k]['idx']))
    for i in range(len(xspace)):
        Q2   = conf['dy-pion tabs'][k]['Q2'][i]
        Jac  = conf['dy-pion tabs'][k]['Jac'][i]
        units= conf['dy-pion tabs'][k]['Units'][i]
        S    = conf['dy-pion tabs'][k]['S'][i]
        Y    = conf['dy-pion tabs'][k]['Y'][i]
        if part!='full': xspace[i]=conf['dy-pion'].get_xsec(Q2,S,Y,Q2,ilum='flavors',part=part,flav=flav) * Jac * units
        elif part=='full': xspace[i]=conf['dy-pion'].get_xsec(Q2,S,Y,Q2,ilum='normal',part=part,flav=flav) * Jac * units
    return xspace

def get_mellspace(k,part,flav=None):
    conf['dy-pion'].load_melltab_hybrid()
    mellspace=np.zeros(len(conf['dy-pion tabs'][k]['idx']))
    for i in range(len(mellspace)):
        Q2   = conf['dy-pion tabs'][k]['Q2'][i]
        Jac  = conf['dy-pion tabs'][k]['Jac'][i]
        units= conf['dy-pion tabs'][k]['Units'][i]
        mellspace[i]=conf['dy-pion'].get_xsec_mell_hybrid(k,i,Q2,part,flav=flav) * Jac * units
    return mellspace

def get_nnspace(k,part,flav=None):
    conf['dy-pion'].load_nn_hybrid()
    nnspace=np.zeros(len(conf['dy-pion tabs'][k]['idx']))
    for i in range(len(nnspace)):
        Q2   = conf['dy-pion tabs'][k]['Q2'][i]
        Jac  = conf['dy-pion tabs'][k]['Jac'][i]
        units= conf['dy-pion tabs'][k]['Units'][i]
        nnspace[i]=conf['dy-pion'].get_xsec_mell_hybrid_nn(k,i,Q2,part,flav=flav) * Jac * units
    return nnspace

In [ ]:
def plot_mellin(mell,nn,k,part,flav,i):
    Z=conf['mellin'].Z
    nrows,ncols=1,2
    fig=py.figure(figsize=(4*ncols,4*nrows))

    ax=py.subplot(nrows,ncols,1)
    ax.plot(Z,mell.real,label='mellin')
    ax.plot(Z,nn.real,label='nn')
    ax.set_title(r'$x_F=%.3f$'%conf['dy-pion tabs'][k]['xF'][i],size=20)
    ax.set_xlabel(r'$Z$')
    ax.set_ylabel(r'Re[$\sigma$]')

    ax=py.subplot(nrows,ncols,2)
    ax.plot(Z,mell.imag,label='mellin')
    ax.plot(Z,nn.imag,label='nn')
    ax.set_title(r'$Q^2=%.3f$'%conf['dy-pion tabs'][k]['Q2'][i],size=20)
    ax.set_xlabel(r'$Z$')
    ax.set_ylabel(r'Im[$\sigma$]')

    ax.legend()

    py.tight_layout()
    if part=='full':
        checkdir('gallery/%s/%s'%(k,part))
        py.savefig('gallery/%s/%s/testmell%s.png'%(k,part,i))
        py.close()
    else:
        checkdir('gallery/%s/%s/%s'%(k,part,flav))
        py.savefig('gallery/%s/%s/%s/testmell%s.png'%(k,part,flav,i))
        py.close()

In [ ]:
def plot(tabs,x,mell,nn,k,part,flav):

    nrows,ncols=1,1
    fig=py.figure(figsize=(ncols*5,nrows*7))
    ax=py.subplot(nrows,ncols,1)

    xF=conf['dy-pion tabs'][k]['xF']
    
    ax.plot(xF,x,label='x-space')
    ax.plot(xF,mell,'*',label='using Mellin tables')
    ax.plot(xF,nn,label='neural net')
    ax.legend()

    ax.set_ylabel(r'$\frac{d\sigma}{dx_F d\sqrt{\tau}}$')
    ax.set_xlabel(r'$x_F$')
    ax.set_title(r'dataset=%s'%k)

    py.tight_layout()
    if part=='full':
        checkdir('gallery/%s/%s'%(k,part))
        py.savefig('gallery/%s/%s/test.png'%(k,part))
        py.close()
    else:
        checkdir('gallery/%s/%s/%s'%(k,part,flav))
        py.savefig('gallery/%s/%s/%s/test.png'%(k,part,flav))
        py.close()

In [ ]:
k=30001
parts=['qA,qbB','qbA,qB','qA,gB','gA,qB']
for part in parts:
    if part=='qA,qbB': flavs=['ub','db','sb','cb','bb']
    if part=='qbA,qB': flavs=['u','d','s','c','b']
    if part=='qA,gB':  flavs=['g']
    if part=='gA,qB':  flavs=['u','ub','d','db','s','sb','c','cb','b','bb']
    for f in flavs:
        xspacedat=get_xspace(k,part,flav=f)
        mellspacedat=get_mellspace(k,part,flav=f)
        nnspacedat=get_nnspace(k,part,flav=f)

        plot(conf['dy-pion tabs'][k],xspacedat,mellspacedat,nnspacedat,k,part,f)
        for i in range(len(conf['dy-pion tabs'][k]['idx'])):
            print('\nplotting the integrands')
            mellinxsec=load('%s/data/%s/%s/%s/%smell.dat'%(conf['path2ml4jam'],k,part,f,i))
            nnxsec=load('%s/data/%s/%s/%s/%snn.dat'%(conf['path2ml4jam'],k,part,f,i))

            plot_mellin(mellinxsec,nnxsec,k,part,f,i)
part='full'
xspacedat=get_xspace(k,part)
mellspacedat=get_mellspace(k,part)
nnspacedat=get_nnspace(k,part)

plot(conf['dy-pion tabs'][k],xspacedat,mellspacedat,nnspacedat,k,part,None)

for i in range(len(conf['dy-pion tabs'][k]['idx'])):
    print('\nplotting the integrands')
    mellinub=load('%s/data/%s/%s/%sxsecmell.dat'%(conf['path2ml4jam'],k,part,i))
    nnub=load('%s/data/%s/%s/%sxsecnn.dat'%(conf['path2ml4jam'],k,part,i))

    plot_mellin(mellinub,nnub,k,part,None,i)